# Voxel VAE-GAN Training

This notebook is designed to provide a wholistic vae-gan training experience. You can adjust the model and training parameters through the sacred configuration file, you can view training progress in tensorboard, and you can (wip) create reconstructions with the saved models!

References:

* https://github.com/anitan0925/vaegan/blob/master/examples/train.py
  * Runs 20 epochs on separate VAE and GAN then 200 on VAEGAN
* https://github.com/jlindsey15/VAEGAN/blob/master/main.py
  * Almost clear code for vaegan paper
* https://arxiv.org/pdf/1512.09300.pdf
  * vaegan paper
* https://github.com/timsainb/Tensorflow-MultiGPU-VAE-GAN
  * Best code yet!
  

## Setup

In [1]:
import env
from train_vaegan import train_vaegan
from data.thingi10k import Thingi10k
from data.modelnet10 import ModelNet10
from data import MODELNET10_INDEX, MODELNET10_TOILET_INDEX, MODELNET10_SOFA_INDEX
from models import MODEL_DIR


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

## Prepare Sacred Experiment

In [2]:
from sacred.observers import FileStorageObserver
from sacred import Experiment
import os

ex = Experiment(name='voxel_vaegan_notebook', interactive=True)
ex.observers.append(FileStorageObserver.create('experiments_vaegan'))

@ex.main
def run_experiment(cfg):
    train_vaegan(cfg)

import datetime
last_model_dir = None

## Prepare Model Config

The model dir is generated with a timestamp. This keeps you from overwriting past results and keeps results separate to avoid confusing tensorboard.

But be warned! These model dirs can take up space, so you might need to periodically go back and delete ones you do not care about.

Also, if you ever train a model that you would really like to keep, I recommend moving it to a new directory with a special name like "best_model_ever".

VAE Training Notes:

* Higher KL weight results in network focusing less on minimizing reconstruction loss and thus produces lower quality reconstructions
* KL weight < .1 sees diminishing gains; use kl == .1
* Reconstruction loss is very important
* Latent dim of 100 borrowed from reference paper
* Latent dim of 1000 results in lower losses but overtrains (also, reconstructions do not look as good)
* TODO: experiment with latent dim of 350
* Learning rate of 0.0001 borrowed from reference papers; 0.001 trains faster but hits nan loss after a while

VAEGAN Training Notes:

* Architecture as-is (after adjusting loss functions from GAN loss to VAEGAN loss)

In [4]:
DATASET_CLASS = 'ModelNet10'
#INDEX = MODELNET10_SOFA_TOILET_INDEX
#INDEX = MODELNET10_SOFA_INDEX
#INDEX = MODELNET10_TOILET_INDEX
INDEX = MODELNET10_INDEX

def make_cfg():
    model_dir = os.path.join(
        MODEL_DIR,
        'voxel_vaegan1/modelnet10/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')))
    print(model_dir)
    os.makedirs(model_dir)

    cfg = {
        'cfg': {
            "dataset": {
                "class": DATASET_CLASS,
                "index": INDEX,
                #"tag": "animal",
                #"filter_id": 126660,
                #"pctile": 1.0,
                "splits": True,
                #'categories': ['monitor', 'night_stand', 'toilet'],
                #"splits": {
                #    "train": .8,
                #    "dev": .1,
                #    "test": .1
                #}
            },
            "generator": {
                "verbose": True,
                "pad": True
            }, 
            "model": {
                "ckpt_dir": model_dir,
                "voxels_dim": 32,
                "batch_size": 128,
                # Do 0.0001 for 1 epoch, then 0.001 for rest of training
                #"learning_rate": [(1, 0.0001), (None, 0.001)],
                #"learning_rate": 0.0001,
                "enc_lr": 0.0001,  # 0.001 hits nan loss eventually
                "dec_lr": 0.0001,
                "dis_lr": 0.0001,
                "epochs": 201,
                "keep_prob": 0.8,
                "kl_div_loss_weight": .1,
                "recon_loss_weight": 10000,
                "ll_weight": 1,
                "dec_weight": 100,
                "latent_dim": 100,
                "verbose": True,
                "debug": True,
                "input_repeats": 1,
                "display_step": 1,
                #"example_stl_id": 126660,
                "voxel_prob_threshold": 0.065,
                "dev_step": 5,
                "save_step": 5,
                'launch_tensorboard': True,
                'tb_dir': 'tb',
                #'tb_compare': [('best_sofa_and_toilet', '/home/jcworkma/jack/3d-form/models/voxel_vaegan1/modelnet10/2019-03-15_17-08-43/tb')],
                #'tb_compare': [('best_vaegan', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29/tb')],
                #'tb_compare': [('vaegan_100epochs_toilets', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53/tb')],
                #'tb_compare': [('vaegan_logless_loss', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-22_16-01-54/tb')],
                #'tb_compare': [('voxel_vae_toilet_200epochs_1', os.path.join(MODEL_DIR, 'voxel_vae_toilet_200epochs_1', 'tb'))],
                #'tb_compare': [('vae_200epochs_p1kl', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-25_11-05-00/tb')],
                'tb_compare': [('vae_toilet_nightstand_monitor', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-27_05-27-14/tb')],
                'no_gan': True,
                'monitor_memory': True,
                # these settings control how often the components' optimizers are executed during the training loop
                'train_vae_cadence': 1,
                'train_gan_cadence': 1,
                'dis_noise': 0,
                'adaptive_lr': False
            }
        }
    }
    
    return cfg

## Tensorboard Prep

We launch tensorboard with a call to the python subprocess module. Sometimes, that process does not die with the rest of the experiment and lingers on as a system process. This becomes a problem when we try to initialize tensorboard for the next experiment because they cannot share the same port!

The function below is designed to solve this problem. It uses the linux pgrep utility to search for existing tensorboard processes and kill them. Note that this probably won't work on Windows.

In [5]:
from utils import kill_tensorboard

kill_tensorboard()

['pgrep', 'tensorboard'] yielded -> b''


## Training

We start with a check that we are not attempting to overwrite the last MODEL_DIR. If you are blocked by the assert, re-execute the cfg code above to generate a new MODEL_DIR. This will allow you to move ahead with training.

The sacred experiment will save away a copy of your experiment settings in an experiments directory. This can be accessed later in case we need to retrieve a prime config.

If tensorboard is enabled, tune in at localhost:6006 or your_ip:6006
   

In [6]:
cfg = make_cfg()
model_dir = cfg.get('cfg').get('model').get('ckpt_dir')
kill_tensorboard()

/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-28_15-09-00
['pgrep', 'tensorboard'] yielded -> b''


In [ ]:
if last_model_dir == model_dir:
    print('dont overwrite!')
    assert False
else:
    last_model_dir = model_dir

ex.run(config_updates=cfg)

WARNING - root - Added new config entry: "cfg.dataset.class"
WARNING - root - Added new config entry: "cfg.dataset.index"
WARNING - root - Added new config entry: "cfg.dataset.splits"
WARNING - root - Added new config entry: "cfg.generator.pad"
WARNING - root - Added new config entry: "cfg.generator.verbose"
WARNING - root - Added new config entry: "cfg.model.adaptive_lr"
WARNING - root - Added new config entry: "cfg.model.batch_size"
WARNING - root - Added new config entry: "cfg.model.ckpt_dir"
WARNING - root - Added new config entry: "cfg.model.debug"
WARNING - root - Added new config entry: "cfg.model.dec_lr"
WARNING - root - Added new config entry: "cfg.model.dec_weight"
WARNING - root - Added new config entry: "cfg.model.dev_step"
WARNING - root - Added new config entry: "cfg.model.dis_lr"
WARNING - root - Added new config entry: "cfg.model.dis_noise"
WARNING - root - Added new config entry: "cfg.model.display_step"
WARNING - root - Added new config entry: "cfg.model.enc_lr"
WARNI

Logging to /home/jcworkma/jack/3d-form/src/logs/2019-03-28_15-09__root.log
Starting train_vaegan main
Numpy random seed: 486546161
Saved cfg: /home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-28_15-09-00/cfg.json
Dataset: <class 'data.modelnet10.ModelNet10'>
Using dataset index /home/jcworkma/jack/3d-form/src/../data/processed/modelnet10_index.csv and pctile None
Shuffling dataset
dataset n_input=78368
Splitting Datasets
Num input = 78368
Num batches per epoch = 612.25
Initializing VoxelVaegan
encoder/enc_conv1/batchnorm/add_1:0: (?, 30, 30, 30, 8)
encoder/batch_normalization/batchnorm/add_1:0: (?, 15, 15, 15, 16)
encoder/batch_normalization_1/batchnorm/add_1:0: (?, 13, 13, 13, 32)
encoder/batch_normalization_2/batchnorm/add_1:0: (?, 7, 7, 7, 64)
encoder/batch_normalization_3/batchnorm/add_1:0: (?, 7, 7, 7, 343)
encoder/dense_1/BiasAdd:0: (?, 100)
encoder/dense_2/BiasAdd:0: (?, 100)
epsilon: (?, 100)
z: (?, 100)
input_z: (?, 100)
decoder/dec_dense1/BiasAdd:0: (

<tf.Variable 'encoder/batch_normalization_2/moving_variance/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_2/moving_variance/Adam_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/dense/kernel/Adam:0' shape=(64, 343) dtype=float32_ref>
<tf.Variable 'encoder/dense/kernel/Adam_1:0' shape=(64, 343) dtype=float32_ref>
<tf.Variable 'encoder/dense/bias/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/dense/bias/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/gamma/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/gamma/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/beta/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/beta/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/moving_mean/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normaliz

<tf.Variable 'decoder/dec_conv4/kernel/Adam_1:0' shape=(4, 4, 4, 8, 16) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/gamma/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/gamma/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/beta/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/beta/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_mean/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_mean/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_variance/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_variance/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/dec_conv5/kernel/Adam:0' shape=(3, 3, 3, 1, 8) dtype=float32_ref>
<tf.Variable 'decoder/dec_conv5/kernel/Adam_1:0' shape=(3, 3, 3, 1, 8) dtype=fl

Enc Loss = 330.71, KL Divergence = 160.88, Reconstruction Loss = 0.03, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 3.62 mins
Memory Use (GB): 1.4318885803222656
Epoch: 0 / 201, Batch: 57 (0 / 7424), Elapsed time: 3.62 mins
Enc Loss = 355.23, KL Divergence = 176.36, Reconstruction Loss = 0.03, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 3.68 mins
Memory Use (GB): 1.3830833435058594
Epoch: 0 / 201, Batch: 58 (0 / 7552), Elapsed time: 3.69 mins
Enc Loss = 349.43, KL Divergence = 140.17, Reconstruction Loss = 0.03, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 3.75 mins
Memory Use (GB): 1.4039497375488281
Epoch: 0 / 201, Batch: 59 (0 / 7680), Elapsed time: 3.75 mins
Enc Loss = 331.89, KL Divergence = 150.82, Reconstruction Loss = 0.03, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 3.81 mins
Memory Use (GB): 1.40863037109375
Epoch: 0 / 201, Batch: 60 (0 / 7808), Elapsed time: 3.81

...

Enc Loss = 74.09, KL Divergence = 190.20, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 645.24 mins
Memory Use (GB): 1.27020263671875
Epoch: 20 / 201, Batch: 174 (22400 / 63744), Elapsed time: 645.24 mins
Enc Loss = 72.62, KL Divergence = 192.02, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 645.30 mins
Memory Use (GB): 1.2352561950683594
Epoch: 20 / 201, Batch: 175 (22528 / 63744), Elapsed time: 645.30 mins
Enc Loss = 71.42, KL Divergence = 186.16, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 645.37 mins
Memory Use (GB): 1.3375244140625
Epoch: 20 / 201, Batch: 176 (22656 / 63744), Elapsed time: 645.37 mins
Enc Loss = 67.65, KL Divergence = 185.35, Reconstruction Loss = 0.00, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 645.43 mins
Memory Use (GB): 1.4181976318359375
Epoch: 20 / 201, Batch: 177 

Memory Use (GB): 1.3795738220214844
Epoch: 20 / 201, Batch: 205 (26368 / 63744), Elapsed time: 647.21 mins
Enc Loss = 69.98, KL Divergence = 186.80, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 647.27 mins
Memory Use (GB): 1.30987548828125
Epoch: 20 / 201, Batch: 206 (26496 / 63744), Elapsed time: 647.27 mins
Enc Loss = 74.47, KL Divergence = 191.71, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 647.33 mins
Memory Use (GB): 1.3492584228515625
Epoch: 20 / 201, Batch: 207 (26624 / 63744), Elapsed time: 647.34 mins
Enc Loss = 74.49, KL Divergence = 190.44, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 647.40 mins
Memory Use (GB): 1.3365745544433594
Epoch: 20 / 201, Batch: 208 (26752 / 63744), Elapsed time: 647.40 mins
Enc Loss = 81.72, KL Divergence = 193.98, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, 

Enc Loss = 71.99, KL Divergence = 195.31, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 649.23 mins
Memory Use (GB): 1.3368110656738281
Epoch: 20 / 201, Batch: 237 (30464 / 63744), Elapsed time: 649.23 mins
Enc Loss = 71.61, KL Divergence = 191.51, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 649.30 mins
Memory Use (GB): 1.3835945129394531
Epoch: 20 / 201, Batch: 238 (30592 / 63744), Elapsed time: 649.30 mins
Enc Loss = 77.03, KL Divergence = 196.41, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 649.36 mins
Memory Use (GB): 1.39837646484375
Epoch: 20 / 201, Batch: 239 (30720 / 63744), Elapsed time: 649.36 mins
Enc Loss = 74.29, KL Divergence = 189.64, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 649.42 mins
Memory Use (GB): 1.2005577087402344
Epoch: 20 / 201, Batch: 2

Memory Use (GB): 1.3709678649902344
Epoch: 20 / 201, Batch: 268 (34432 / 63744), Elapsed time: 651.20 mins
Enc Loss = 79.24, KL Divergence = 194.96, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 651.26 mins
Memory Use (GB): 1.2969741821289062
Epoch: 20 / 201, Batch: 269 (34560 / 63744), Elapsed time: 651.26 mins
Enc Loss = 72.09, KL Divergence = 188.22, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 651.32 mins
Memory Use (GB): 1.47186279296875
Epoch: 20 / 201, Batch: 270 (34688 / 63744), Elapsed time: 651.33 mins
Enc Loss = 73.25, KL Divergence = 191.98, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 651.39 mins
Memory Use (GB): 1.3821983337402344
Epoch: 20 / 201, Batch: 271 (34816 / 63744), Elapsed time: 651.39 mins
Enc Loss = 75.37, KL Divergence = 190.33, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, 

Enc Loss = 70.24, KL Divergence = 187.80, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 653.22 mins
Memory Use (GB): 1.2644195556640625
Epoch: 20 / 201, Batch: 300 (38528 / 63744), Elapsed time: 653.22 mins
Enc Loss = 73.31, KL Divergence = 195.33, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 653.28 mins
Memory Use (GB): 1.3681182861328125
Epoch: 20 / 201, Batch: 301 (38656 / 63744), Elapsed time: 653.28 mins
Enc Loss = 77.74, KL Divergence = 193.46, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 653.35 mins
Memory Use (GB): 1.295806884765625
Epoch: 20 / 201, Batch: 302 (38784 / 63744), Elapsed time: 653.35 mins
Enc Loss = 71.59, KL Divergence = 192.93, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 653.41 mins
Memory Use (GB): 1.434814453125
Epoch: 20 / 201, Batch: 303 